## Error Analysis for CAD Tag Predictions

In [2]:
import os
import string
import random
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt

In [3]:
# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


In [4]:
import numpy as np

### Test LABELS for TOKENS in TEST Dataset against BERT Outputs

BERT Classifier has returned results for the tokens passed in 'test.tsv' file.  The returned values are probabilities, that need to be converted into equivalent class labels based on majority class.  Then, the class label should be compared against the actual label from the code above to extract the IO-Coding from the xml files.  This is a brute-force approach or a manual way of verifying the validity of the predictions


Read in results from BERT Predicitons to the above dataset
The above dataset is derived from IO-Coding applied as done on the training set. This is what should be based on the annotation process. Nowe, we have to read in the predictions from bert, which is a set of class probabilities across all 3 classes and we have to merget that with the above dataset for comparison and error analysis.

### Data File Names

* Test files with Labels and Filenames : /data_for_bert_sent/test_files_with_labels/*_testfile.csv
* Bert label mapping /data_for_bert_sent/test_files_with_labels/*_labelmapping.csv
* BERT evaluation /data_for_bert_sent/BERT_run_results/*_eval_results.txt


In [5]:
print(os.path.dirname(os.path.abspath('__file__')))

C:\Users\Kalyan\Documents\Anu\W266 - NLP\Final Project\lheart-disease-risk-prediction\Code


In [ ]:
# read in the test files with labels

HI_test = pd.read_csv("data_for_bert_sent/test_files_with_labels/hypertension_ind_testfile.csv")

In [ ]:
HI_test.head(10)

In [ ]:
# read in the test results captured for BERT Hypertension model and specify columns as the actual file has no header
bert_HI_results = pd.read_csv("data_for_bert_sent/BERT_run_results/sentence_based_output_data_hypertension_output_results_test_results.tsv", sep='\t',header=None)
 
bert_HI_results.columns=["Class0", "Class1", "Class2"]

In [ ]:
bert_HI_results.head()

In [ ]:
bert_HI_results['predClass'] = bert_HI_results.idxmax(axis=1)

In [ ]:
bert_HI_results.head()

In [ ]:
bert_HI_results['predClass'].value_counts()

In [ ]:
def HI_set_labels(classlabel):
    if (classlabel=='Class1'):
        return 'high bp'
    elif (classlabel=='Class2'):
        return 'mention'
    else:
        return 'Other'

In [ ]:
bert_HI_results['predLabel'] = bert_HI_results['predClass'].apply(HI_set_labels)

bert_HI_results.head(10)


In [ ]:
# validating the counts by label
bert_HI_results['predLabel'].value_counts()

In [ ]:
HI_combined = pd.concat([HI_test, bert_HI_results], axis=1)

In [ ]:
HI_combined.head()

In [ ]:
HI_combined[HI_combined['predLabel']!='Other']

In [ ]:
HI_test_labels = HI_combined['label']
HI_pred_labels = HI_combined['predLabel']

#print(type(HI_test_labels))

In [ ]:
accuracy_score(HI_test_labels, HI_pred_labels)

In [ ]:
print(classification_report(HI_pred_labels, HI_test_labels))

In [ ]:
unique_label = np.unique(HI_test_labels)
print(pd.DataFrame(confusion_matrix(HI_test_labels, HI_pred_labels, labels=unique_label), 
                   index=['true:{:}'.format(x) for x in unique_label], 
                   columns=['pred:{:}'.format(x) for x in unique_label]))

In [ ]:
from sklearn.utils.multiclass import unique_labels

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    #classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")
    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")

    fig.tight_layout()
    return ax

plot_confusion_matrix(HI_test_labels, HI_pred_labels, ["Other", "high bp", "mention"])
plt.show()

In [ ]:
# read in the test files with labels

DT_test = pd.read_csv("data_for_bert_sent/test_files_with_labels/diabetes_time_testfile.csv")

In [ ]:
DT_test.head(10)

In [ ]:
# read in the test results captured for BERT Hypertension model and specify columns as the actual file has no header
bert_DT_results = pd.read_csv("data_for_bert_sent/BERT_run_results/diabetes_time_test_results.tsv", sep='\t',header=None)
 
bert_DT_results.columns=["Class0", "Class1", "Class2", "Class3"]

In [ ]:
bert_DT_results.head()

In [ ]:
bert_DT_results['predClass'] = bert_DT_results.idxmax(axis=1)

In [ ]:
bert_DT_results.head()

In [ ]:
bert_DT_results['predClass'].value_counts()

In [ ]:
def DT_set_labels(classlabel):
    if (classlabel=='Class1'):
        return 'after DCT'
    elif (classlabel=='Class2'):
        return 'before DCT'
    elif (classlabel=='Class3'):
        return 'during DCT'
    else:
        return 'Other'

In [ ]:
bert_DT_results['predLabel'] = bert_DT_results['predClass'].apply(DT_set_labels)

bert_DT_results.head(10)


In [ ]:
# validating the counts by label
bert_DT_results['predLabel'].value_counts()

In [ ]:
DT_combined = pd.concat([DT_test, bert_DT_results], axis=1)

In [ ]:
DT_combined.head()

In [ ]:
DT_combined[DT_combined['predLabel']!='Other']

In [ ]:
DT_test_labels = DT_combined['label']
DT_pred_labels = DT_combined['predLabel']

#print(type(DT_test_labels))

In [ ]:
accuracy_score(DT_test_labels, DT_pred_labels)

In [ ]:
print(classification_report(DT_pred_labels, DT_test_labels))

In [ ]:
print("Confusion Matrix \n", confusion_matrix(DT_test_labels, DT_pred_labels))

In [ ]:
unique_label = np.unique(DT_test_labels)
print(pd.DataFrame(confusion_matrix(DT_test_labels, DT_pred_labels, labels=unique_label), 
                   index=['true:{:}'.format(x) for x in unique_label], 
                   columns=['pred:{:}'.format(x) for x in unique_label]))

### Interpreting the predictions compared against actual test labels

As seen above, BERT predictions seem very accurate and it seems to predict only after it has seen the complete context.  Also, punctuation marks are not labeled as one of the relevant classes, although a human annotator has done based on the instructions provided as part of the annotation process.  